In [1]:
!pip install fake-useragent

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import os
from fake_useragent import UserAgent

In [124]:
cities = ["tehran",
        "hamedan",
        "isfahan",
        "sanandaj",
        "mashhad",
        "bushehr",
        "tabriz",
        "rasht",
        "shiraz"]
links = [ 
            "buy-office?has-photo=true", 
             "buy-villa?has-photo=true",
             "buy-old-house?has-photo=true", 
            "car?has-photo=true&exchange=exclude-exchanges",
            "bike-skate-scooter",
            "buy-apartment?has-photo=true",
            "computers",
            "mobile-phones?has-photo=true&exchange=exclude-exchanges", 
            "laptop-notebook-macbook?has-photo=true&exchange=exclude-exchanges", 
            "motorcycles?has-photo=true&exchange=exclude-exchanges", 
            "refrigerator-freezer?has-photo=true&exchange=exclude-exchanges",
            "washing-machines?has-photo=true&exchange=exclude-exchanges",
            "dishwasher"]

urls = []
for link in links:
    for city in  cities :
        urls.append(f"https://divar.ir/s/{city}/{link}")

In [149]:
tokens=[]
token_links=[]
for url in tqdm(urls):
    page = requests.get(url)
    bs = BeautifulSoup(page.text, 'html.parser')
    data = bs.find_all("div", class_="post-card-item-af972 kt-col-6-bee95 kt-col-xxl-4-e9d46")
    for tag in data:
        token = tag.find('a',attrs={'class':''})['href']
        tokens.append(token)
        cat1= url.split("/")[-1].replace("&exchange=exclude-exchanges", "").replace("?has-photo=true", "")
        token_links.append( [cat1 ,f"https://divar.ir{token}"])

100%|██████████| 117/117 [02:00<00:00,  1.03s/it]


In [150]:
token_links[0]

['buy-office',
 'https://divar.ir/v/آرژانتین-بخارست-۲۱۰-متر-سند-اداری-۶-اتاق-تکواحدی_دفتر-کار-اتاق-اداری-و-مطب_تهران_آرژانتین_دیوار/AZ9ogD_B']

In [151]:
df = pd.DataFrame (token_links, columns = ['cat3', 'link'])
print (df)

            cat3                                               link
0     buy-office  https://divar.ir/v/آرژانتین-بخارست-۲۱۰-متر-سند...
1     buy-office  https://divar.ir/v/آرژانتین-بخارست-۱۳۵-متر-مبل...
2     buy-office  https://divar.ir/v/215متر-سند-اداری-موقعیت-تجا...
3     buy-office  https://divar.ir/v/145مترسنداداری-2پارکینگ-بر-...
4     buy-office  https://divar.ir/v/سعادت-آباد-۱۰۵متر-نوساز-فول...
...          ...                                                ...
2751  dishwasher  https://divar.ir/v/ماشین-ظرفشویی-بوش-آلمان-سری...
2752  dishwasher  https://divar.ir/v/ماشین-ظرفشویی-بوش_ماشین-ظرف...
2753  dishwasher  https://divar.ir/v/خریدار-ماشین-ظرفشویی-هستم-د...
2754  dishwasher  https://divar.ir/v/ماشین-ظرفشویی-۱۴نفره-جی-پلا...
2755  dishwasher  https://divar.ir/v/ماشین-ظرفشویی-دوو-مدل-dwk-2...

[2756 rows x 2 columns]


In [153]:
df.to_csv("links.csv")

In [154]:
df['cat3'].unique()

array(['buy-office', 'buy-villa', 'buy-old-house', 'car',
       'bike-skate-scooter', 'buy-apartment', 'computers',
       'mobile-phones', 'laptop-notebook-macbook', 'motorcycles',
       'refrigerator-freezer', 'washing-machines', 'dishwasher'],
      dtype=object)

In [155]:
df1 = df[df['cat3']== 'washing-machines' ]

In [156]:
df1 = df1.reset_index()

In [163]:
all_data = []
ua = UserAgent()
for i in tqdm(range(len(df1))):
    try:
        url = url.strip()
        url = df1['link'][i]
        time.sleep(np.random.randint(50,1000)/1000)
        header = {'User-Agent':str(ua.random)}
        page = requests.get(url)
        bs = BeautifulSoup(page.text, 'html.parser')
        data = {"cat1": "home-kitchen", 
                    "cat2": "appliance", 
                    "cat3": "washing-machines", 
                    "title": bs.find_all("div", class_="kt-page-title__title kt-page-title__title--responsive-sized")[0].text, 
                    "description": bs.find_all("p", class_="kt-description-row__text kt-description-row__text--primary")[0].text, 
                    "url": url, 
                    "sub_title": bs.find_all("div", class_="kt-page-title__subtitle kt-page-title__subtitle--responsive-sized")[0].text,
                    "price": bs.find_all("p", class_="kt-unexpandable-row__value")[-1].text.replace(" تومان", "").replace("(مقطوع)", "").replace("توافقی", "NAN"),
                    "size": None,
                    "year": bs.find_all("span", class_="kt-group-row-item__value")[1].text if soup.find_all("span", class_="kt-group-row-item__value") else None, 
                    "room_number": None, 
                    "color": bs.find_all("span", class_="kt-group-row-item__value")[2].text if len(soup.find_all("span", class_="kt-group-row-item__value")) > 2 else None, 
                    "mileage": bs.find_all("span", class_="kt-group-row-item__value")[0].text if soup.find_all("span", class_="kt-group-row-item__value") else None,
                    "CPU": None,
                    "RAM": None
                   }
        all_data.append(data)
    
    except:
        pass  
    

100%|██████████| 216/216 [04:34<00:00,  1.27s/it]


In [165]:
pd.json_normalize(all_data).to_csv('washing-machines.csv')

In [200]:
df2 = df[df['cat3']== 'buy-apartment' ]
df2 = df2.reset_index()

all_data_for_buy_apartment = []

ua = UserAgent()
for i in tqdm(range(len(df2))):
    try:
        url = url.strip()
        url = df2['link'][i]
        time.sleep(np.random.randint(50,1000)/1000)
        header = {'User-Agent':str(ua.random)}
        page = requests.get(url)
        bs = BeautifulSoup(page.text, 'html.parser')
        data = {"cat1": "real-estate", 
                "cat2": "buy-residential", 
                "cat3": "buy-apartment", 
                "title": bs.find_all("div", class_="kt-page-title__title kt-page-title__title--responsive-sized")[0].text, 
                "description": bs.find_all("p", class_="kt-description-row__text kt-description-row__text--primary")[0].text, 
                "url": url, 
                "sub_title": bs.find_all("div", class_="kt-page-title__subtitle kt-page-title__subtitle--responsive-sized")[0].text,
                "price": bs.find_all("p", class_="kt-unexpandable-row__value")[0].text.replace(" تومان", "").replace("(مقطوع)", "").replace("توافقی", "NAN"),  
                "size": bs.find_all("span", class_="kt-group-row-item__value")[0].text.replace(" متر", ""),
                "year": bs.find_all("span", class_="kt-group-row-item__value")[1].text.replace("قبل از ", ""), 
                "room_number": bs.find_all("span", class_="kt-group-row-item__value")[2].text.replace("بدون اتاق", "۰"), 
                "color": None, 
                "mileage": None,
                "CPU": None,
                "RAM": None
               }
        all_data_for_buy_apartment.append(data)
    
    except:
        pass  
pd.json_normalize(all_data_for_buy_apartment).to_csv('buy-apartment.csv')  
print(len(all_data_for_buy_apartment))

100%|██████████| 216/216 [04:46<00:00,  1.33s/it]

216


In [201]:
df3 = df[df['cat3']== 'buy-villa' ]
df3 = df3.reset_index()
all_data_for_buy_villa = []

ua = UserAgent()
for i in tqdm(range(len(df3))):
    try:
        url = url.strip()
        url = df3['link'][i]
        time.sleep(np.random.randint(50,1000)/1000)
        header = {'User-Agent':str(ua.random)}
        page = requests.get(url)
        bs = BeautifulSoup(page.text, 'html.parser')
        data = {"cat1": "real-estate", 
                "cat2": "buy-residential", 
                "cat3": "buy-apartment", 
                "title": bs.find_all("div", class_="kt-page-title__title kt-page-title__title--responsive-sized")[0].text, 
                "description": bs.find_all("p", class_="kt-description-row__text kt-description-row__text--primary")[0].text, 
                "url": url, 
                "sub_title": bs.find_all("div", class_="kt-page-title__subtitle kt-page-title__subtitle--responsive-sized")[0].text,
                "price": bs.find_all("p", class_="kt-unexpandable-row__value")[0].text.replace(" تومان", "").replace("(مقطوع)", "").replace("توافقی", "NAN"),  
                "size": bs.find_all("span", class_="kt-group-row-item__value")[0].text.replace(" متر", ""),
                "year": bs.find_all("span", class_="kt-group-row-item__value")[1].text.replace("قبل از ", ""), 
                "room_number": bs.find_all("span", class_="kt-group-row-item__value")[2].text.replace("بدون اتاق", "۰"), 
                "color": None, 
                "mileage": None,
                "CPU": None,
                "RAM": None
               }
        all_data_for_buy_villa.append(data)
    
    except:
        pass  
pd.json_normalize(all_data_for_buy_villa).to_csv('buy-villa.csv')  
print(len(all_data_for_buy_villa))

100%|██████████| 216/216 [04:45<00:00,  1.32s/it]

213


In [202]:
df4 = df[df['cat3']== 'buy-office' ]
df4 = df4.reset_index()
all_data_for_buy_office = []

ua = UserAgent()
for i in tqdm(range(len(df4))):
    try:
        url = url.strip()
        url = df4['link'][i]
        time.sleep(np.random.randint(50,1000)/1000)
        header = {'User-Agent':str(ua.random)}
        page = requests.get(url)
        bs = BeautifulSoup(page.text, 'html.parser')
        data = {"cat1": "real-estate", 
                "cat2": "buy-commercial-property", 
                "cat3": "buy-office", 
                "title": bs.find_all("div", class_="kt-page-title__title kt-page-title__title--responsive-sized")[0].text, 
                "description": bs.find_all("p", class_="kt-description-row__text kt-description-row__text--primary")[0].text, 
                "url": url, 
                "sub_title": bs.find_all("div", class_="kt-page-title__subtitle kt-page-title__subtitle--responsive-sized")[0].text,
                "price": bs.find_all("p", class_="kt-unexpandable-row__value")[0].text.replace(" تومان", "").replace("(مقطوع)", "").replace("توافقی", "NAN"),
                "size": bs.find_all("span", class_="kt-group-row-item__value")[0].text.replace(" متر", ""),
                "year": bs.find_all("span", class_="kt-group-row-item__value")[1].text.replace("قبل از ", ""), 
                "room_number": bs.find_all("span", class_="kt-group-row-item__value")[2].text.replace("بدون اتاق", "۰"), 
                "color": None, 
                "mileage": None,
                "CPU": None,
                "RAM": None
               }
        all_data_for_buy_office.append(data)
    
    except:
        pass  
pd.json_normalize(all_data_for_buy_office).to_csv('buy-office.csv') 
print(len(all_data_for_buy_office))

100%|██████████| 188/188 [04:04<00:00,  1.30s/it]

187


In [203]:
df5 = df[df['cat3']== 'car' ]
df5 = df5.reset_index()
all_data_for_car = []

ua = UserAgent()
for i in tqdm(range(len(df5))):
    try:
        url = url.strip()
        url = df5['link'][i]
        time.sleep(np.random.randint(50,1000)/1000)
        header = {'User-Agent':str(ua.random)}
        page = requests.get(url)
        bs = BeautifulSoup(page.text, 'html.parser')
        data = {"cat1": "vehicles", 
                "cat2": "auto", 
                "cat3": "car", 
                "title": bs.find_all("div", class_="kt-page-title__title kt-page-title__title--responsive-sized")[0].text, 
                "description": bs.find_all("p", class_="kt-description-row__text kt-description-row__text--primary")[0].text, 
                "url": url, 
                "sub_title": bs.find_all("div", class_="kt-page-title__subtitle kt-page-title__subtitle--responsive-sized")[0].text,
                "price": bs.find_all("p", class_="kt-unexpandable-row__value")[-1].text.replace(" تومان", "").replace("(مقطوع)", "").replace("توافقی", "NAN"),
                "size": None,
                "year": bs.find_all("span", class_="kt-group-row-item__value")[1].text if soup.find_all("span", class_="kt-group-row-item__value") else None, 
                "room_number": None, 
                "color": bs.find_all("span", class_="kt-group-row-item__value")[2].text if len(soup.find_all("span", class_="kt-group-row-item__value")) > 2 else None, 
                "mileage": bs.find_all("span", class_="kt-group-row-item__value")[0].text if soup.find_all("span", class_="kt-group-row-item__value") else None,
                "CPU": None,
                "RAM": None
               }
        all_data_for_car.append(data)
    
    except:
        pass  
pd.json_normalize(all_data_for_car).to_csv('car.csv')
print(len(all_data_for_car))

100%|██████████| 192/192 [04:09<00:00,  1.30s/it]

192
